In [ ]:
!ls /content/drive/MyDrive/collab


ls: cannot access '/content/drive/MyDrive/collab': No such file or directory


# Тема: Очищення даних та інженерія ознак
# Мета: Підготувати набір даних підприємства до побудови моделей
# машинного навчання — очистити, трансформувати та закодувати дані,
# сформувати повний набір якісних ознак.

# Завдання:

1. Здійсніть підготовку даних вашого підприємства до подальшого
моделювання використовуючи python/ jupyter:

  a. Замініть відсутні значення, якщо такі присутні та обґрунтуйте вибір
методу(ів)

  b. Проведіть кодування (Categorical Encoding) категоріальних ознак

  c. Опрацюйте аномальні значення

  d. Використайте підходи масштабування ознак якщо це доцільно для
ваших даних

2. Сформуйте “Звіт про очищення даних”

3. Обґрунтуйте методи інженерії даних, які були використані для підготовки
даних вашого підприємства.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import chardet
from sqlalchemy import create_engine
import psycopg2



file_path = "/content/drive/MyDrive/collab/customer_booking.csv"
with open(file_path, "rb") as f:
    result = chardet.detect(f.read(50000))
print(result)

dataset = pd.read_csv(file_path, encoding='Windows-1252')
print("Розмірність датасету:", dataset.shape)

dataset.info()
dataset = dataset.drop_duplicates()
dataset.info()






{'encoding': 'Windows-1252', 'confidence': 0.73, 'language': ''}
Розмірність датасету: (50000, 14)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 14 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   num_passengers         50000 non-null  int64  
 1   sales_channel          50000 non-null  object 
 2   trip_type              50000 non-null  object 
 3   purchase_lead          50000 non-null  int64  
 4   length_of_stay         50000 non-null  int64  
 5   flight_hour            50000 non-null  int64  
 6   flight_day             50000 non-null  object 
 7   route                  50000 non-null  object 
 8   booking_origin         50000 non-null  object 
 9   wants_extra_baggage    50000 non-null  int64  
 10  wants_preferred_seat   50000 non-null  int64  
 11  wants_in_flight_meals  50000 non-null  int64  
 12  flight_duration        50000 non-null  float64
 13  booking

In [ ]:
dataset.head()


,num_passengers,trip_type,purchase_lead,length_of_stay,flight_hour,flight_day,route,booking_origin,wants_extra_baggage,wants_preferred_seat,wants_in_flight_meals,flight_duration,booking_complete,sales_channel_Internet,sales_channel_Mobile
0,2,RoundTrip,262,19,7,Sat,AKLDEL,New Zealand,1,0,0,5.52,0,1,0
1,1,RoundTrip,112,20,3,Sat,AKLDEL,New Zealand,0,0,0,5.52,0,1,0
2,2,RoundTrip,243,22,17,Wed,AKLDEL,India,1,1,0,5.52,0,1,0
3,1,RoundTrip,96,31,4,Sat,AKLDEL,New Zealand,0,0,1,5.52,0,1,0
4,2,RoundTrip,68,22,15,Wed,AKLDEL,India,1,0,1,5.52,0,1,0


In [ ]:
for column in dataset.columns:
    unique_values = dataset[column].unique()
    unique_count = dataset[column].nunique()
    print(f"\n🔹 {column} ({unique_count} унікальних значень):")

    print(unique_values)



🔹 num_passengers (9 унікальних значень):
[2 1 3 4 6 5 7 9 8]

🔹 sales_channel (2 унікальних значень):
['Internet' 'Mobile']

🔹 trip_type (3 унікальних значень):
['RoundTrip' 'CircleTrip' 'OneWay']

🔹 purchase_lead (470 унікальних значень):
[262 112 243  96  68   3 201 238  80 378 185   8 265 245 192 259  19  67
 351 228  21  23  25  16  66  51  37 101   7   5  33  20  56  13  71 346
   9  65 221 107  14  77  24  47  22  11  35 272 250 161 128 190 311  27
 314 102   2  26  32  57 198 120  69 281 282  15  31 316 232 156 196 283
  34  63   6 227 181 138 134  85  93 118  90 168 127  18 153 186 263 172
 230  42  81 140 285 179 341 173 180 366 229 174  82  75 163 105 276 110
  83 154  55  45  54 193  10 166  97 239 182  99  53  28 384  70 126 142
  87  86 313  41  91 206 197 268 157 145 241 139  59 396 280  79 380 132
 121 419  58 115 236 225 116 191 275 417 159  92 106 218 295  12 305 235
 332 125 204 310 368 169 214 155 325 130 189 226 122  88 211 188 129 149
  38 194 183  40  95 135 222 

In [ ]:
dataset = pd.get_dummies(dataset, columns=['sales_channel'], prefix='sales_channel')

dummy_cols = [col for col in dataset.columns if col.startswith('sales_channel_')]
dataset[dummy_cols] = dataset[dummy_cols].astype(int)

dataset.info()

<class 'pandas.core.frame.DataFrame'>
Index: 49281 entries, 0 to 49999
Data columns (total 15 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   num_passengers          49281 non-null  int64  
 1   trip_type               49281 non-null  object 
 2   purchase_lead           49281 non-null  int64  
 3   length_of_stay          49281 non-null  int64  
 4   flight_hour             49281 non-null  int64  
 5   flight_day              49281 non-null  object 
 6   route                   49281 non-null  object 
 7   booking_origin          49281 non-null  object 
 8   wants_extra_baggage     49281 non-null  int64  
 9   wants_preferred_seat    49281 non-null  int64  
 10  wants_in_flight_meals   49281 non-null  int64  
 11  flight_duration         49281 non-null  float64
 12  booking_complete        49281 non-null  int64  
 13  sales_channel_Internet  49281 non-null  int64  
 14  sales_channel_Mobile    49281 non-null  int

In [ ]:
dataset.head()


,num_passengers,trip_type,purchase_lead,length_of_stay,flight_hour,flight_day,route,booking_origin,wants_extra_baggage,wants_preferred_seat,wants_in_flight_meals,flight_duration,booking_complete,sales_channel_Internet,sales_channel_Mobile
0,2,RoundTrip,262,19,7,Sat,AKLDEL,New Zealand,1,0,0,5.52,0,1,0
1,1,RoundTrip,112,20,3,Sat,AKLDEL,New Zealand,0,0,0,5.52,0,1,0
2,2,RoundTrip,243,22,17,Wed,AKLDEL,India,1,1,0,5.52,0,1,0
3,1,RoundTrip,96,31,4,Sat,AKLDEL,New Zealand,0,0,1,5.52,0,1,0
4,2,RoundTrip,68,22,15,Wed,AKLDEL,India,1,0,1,5.52,0,1,0


In [ ]:
dataset = pd.get_dummies(dataset, columns=['trip_type'], prefix='trip_type')

dummy_cols = [col for col in dataset.columns if col.startswith('trip_type')]
dataset[dummy_cols] = dataset[dummy_cols].astype(int)

dataset.info()
dataset.head()


<class 'pandas.core.frame.DataFrame'>
Index: 49281 entries, 0 to 49999
Data columns (total 17 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   num_passengers          49281 non-null  int64  
 1   purchase_lead           49281 non-null  int64  
 2   length_of_stay          49281 non-null  int64  
 3   flight_hour             49281 non-null  int64  
 4   flight_day              49281 non-null  object 
 5   route                   49281 non-null  object 
 6   booking_origin          49281 non-null  object 
 7   wants_extra_baggage     49281 non-null  int64  
 8   wants_preferred_seat    49281 non-null  int64  
 9   wants_in_flight_meals   49281 non-null  int64  
 10  flight_duration         49281 non-null  float64
 11  booking_complete        49281 non-null  int64  
 12  sales_channel_Internet  49281 non-null  int64  
 13  sales_channel_Mobile    49281 non-null  int64  
 14  trip_type_CircleTrip    49281 non-null  int

In [ ]:
freq_encoding = dataset['flight_day'].value_counts() / len(dataset)

dataset['flight_day'] = dataset['flight_day'].map(freq_encoding)


In [ ]:
dataset.info()
dataset.head()

<class 'pandas.core.frame.DataFrame'>
Index: 49281 entries, 0 to 49999
Data columns (total 17 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   num_passengers          49281 non-null  int64  
 1   purchase_lead           49281 non-null  int64  
 2   length_of_stay          49281 non-null  int64  
 3   flight_hour             49281 non-null  int64  
 4   flight_day              49281 non-null  float64
 5   route                   49281 non-null  object 
 6   booking_origin          49281 non-null  object 
 7   wants_extra_baggage     49281 non-null  int64  
 8   wants_preferred_seat    49281 non-null  int64  
 9   wants_in_flight_meals   49281 non-null  int64  
 10  flight_duration         49281 non-null  float64
 11  booking_complete        49281 non-null  int64  
 12  sales_channel_Internet  49281 non-null  int64  
 13  sales_channel_Mobile    49281 non-null  int64  
 14  trip_type_CircleTrip    49281 non-null  int

,num_passengers,purchase_lead,length_of_stay,flight_hour,flight_day,route,booking_origin,wants_extra_baggage,wants_preferred_seat,wants_in_flight_meals,flight_duration,booking_complete,sales_channel_Internet,sales_channel_Mobile,trip_type_CircleTrip,trip_type_OneWay,trip_type_RoundTrip
0,2,262,19,7,0.116130,AKLDEL,New Zealand,1,0,0,5.52,0,1,0,0,0,1
1,1,112,20,3,0.116130,AKLDEL,New Zealand,0,0,0,5.52,0,1,0,0,0,1
2,2,243,22,17,0.153447,AKLDEL,India,1,1,0,5.52,0,1,0,0,0,1
3,1,96,31,4,0.116130,AKLDEL,New Zealand,0,0,1,5.52,0,1,0,0,0,1
4,2,68,22,15,0.153447,AKLDEL,India,1,0,1,5.52,0,1,0,0,0,1


In [ ]:
unique_values = dataset['flight_day'].unique()
print(unique_values)


[0.11612995 0.15344656 0.14859682 0.16209087 0.13071975 0.15336539
 0.13565066]


In [ ]:
freq_encoding = dataset['route'].value_counts() / len(dataset)

dataset['route'] = dataset['route'].map(freq_encoding)

dataset.info()
unique_route = dataset['route'].unique()
print(unique_route)

<class 'pandas.core.frame.DataFrame'>
Index: 49281 entries, 0 to 49999
Data columns (total 17 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   num_passengers          49281 non-null  int64  
 1   purchase_lead           49281 non-null  int64  
 2   length_of_stay          49281 non-null  int64  
 3   flight_hour             49281 non-null  int64  
 4   flight_day              49281 non-null  float64
 5   route                   49281 non-null  float64
 6   booking_origin          49281 non-null  object 
 7   wants_extra_baggage     49281 non-null  int64  
 8   wants_preferred_seat    49281 non-null  int64  
 9   wants_in_flight_meals   49281 non-null  int64  
 10  flight_duration         49281 non-null  float64
 11  booking_complete        49281 non-null  int64  
 12  sales_channel_Internet  49281 non-null  int64  
 13  sales_channel_Mobile    49281 non-null  int64  
 14  trip_type_CircleTrip    49281 non-null  int

In [ ]:
dataset.head()


,num_passengers,purchase_lead,length_of_stay,flight_hour,flight_day,route,booking_origin,wants_extra_baggage,wants_preferred_seat,wants_in_flight_meals,flight_duration,booking_complete,sales_channel_Internet,sales_channel_Mobile,trip_type_CircleTrip,trip_type_OneWay,trip_type_RoundTrip
0,2,262,19,7,0.116130,0.000406,New Zealand,1,0,0,5.52,0,1,0,0,0,1
1,1,112,20,3,0.116130,0.000406,New Zealand,0,0,0,5.52,0,1,0,0,0,1
2,2,243,22,17,0.153447,0.000406,India,1,1,0,5.52,0,1,0,0,0,1
3,1,96,31,4,0.116130,0.000406,New Zealand,0,0,1,5.52,0,1,0,0,0,1
4,2,68,22,15,0.153447,0.000406,India,1,0,1,5.52,0,1,0,0,0,1


In [ ]:
freq_encoding = dataset['booking_origin'].value_counts() / len(dataset)

dataset['booking_origin'] = dataset['booking_origin'].map(freq_encoding)

dataset.info()
unique_route = dataset['booking_origin'].unique()
print(unique_route)

<class 'pandas.core.frame.DataFrame'>
Index: 49281 entries, 0 to 49999
Data columns (total 17 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   num_passengers          49281 non-null  int64  
 1   purchase_lead           49281 non-null  int64  
 2   length_of_stay          49281 non-null  int64  
 3   flight_hour             49281 non-null  int64  
 4   flight_day              49281 non-null  float64
 5   route                   49281 non-null  float64
 6   booking_origin          49281 non-null  float64
 7   wants_extra_baggage     49281 non-null  int64  
 8   wants_preferred_seat    49281 non-null  int64  
 9   wants_in_flight_meals   49281 non-null  int64  
 10  flight_duration         49281 non-null  float64
 11  booking_complete        49281 non-null  int64  
 12  sales_channel_Internet  49281 non-null  int64  
 13  sales_channel_Mobile    49281 non-null  int64  
 14  trip_type_CircleTrip    49281 non-null  int

In [ ]:
dataset.head()


,num_passengers,purchase_lead,length_of_stay,flight_hour,flight_day,route,booking_origin,wants_extra_baggage,wants_preferred_seat,wants_in_flight_meals,flight_duration,booking_complete,sales_channel_Internet,sales_channel_Mobile,trip_type_CircleTrip,trip_type_OneWay,trip_type_RoundTrip
0,2,262,19,7,0.116130,0.000406,0.021509,1,0,0,5.52,0,1,0,0,0,1
1,1,112,20,3,0.116130,0.000406,0.021509,0,0,0,5.52,0,1,0,0,0,1
2,2,243,22,17,0.153447,0.000406,0.025527,1,1,0,5.52,0,1,0,0,0,1
3,1,96,31,4,0.116130,0.000406,0.021509,0,0,1,5.52,0,1,0,0,0,1
4,2,68,22,15,0.153447,0.000406,0.025527,1,0,1,5.52,0,1,0,0,0,1


In [ ]:
for column in dataset.columns:
    unique_values = dataset[column].unique()
    unique_count = dataset[column].nunique()
    print(f"\n🔹 {column} ({unique_count} унікальних значень):")

    print(unique_values)



🔹 num_passengers (9 унікальних значень):
[2 1 3 4 6 5 7 9 8]

🔹 purchase_lead (470 унікальних значень):
[262 112 243  96  68   3 201 238  80 378 185   8 265 245 192 259  19  67
 351 228  21  23  25  16  66  51  37 101   7   5  33  20  56  13  71 346
   9  65 221 107  14  77  24  47  22  11  35 272 250 161 128 190 311  27
 314 102   2  26  32  57 198 120  69 281 282  15  31 316 232 156 196 283
  34  63   6 227 181 138 134  85  93 118  90 168 127  18 153 186 263 172
 230  42  81 140 285 179 341 173 180 366 229 174  82  75 163 105 276 110
  83 154  55  45  54 193  10 166  97 239 182  99  53  28 384  70 126 142
  87  86 313  41  91 206 197 268 157 145 241 139  59 396 280  79 380 132
 121 419  58 115 236 225 116 191 275 417 159  92 106 218 295  12 305 235
 332 125 204 310 368 169 214 155 325 130 189 226 122  88 211 188 129 149
  38 194 183  40  95 135 222  64 162 150  73 133  89 151 349  29 123 298
 195  84 410 216 111 109   0 210 326 240  39 294 304 160 363 231  62   1
 178 253  30 199 17

In [ ]:
from sklearn.preprocessing import StandardScaler
import numpy as np

num_passengers = np.array([2, 1, 3, 4, 6, 5, 7, 9, 8]).reshape(-1, 1)
purchase_lead = np.array([262, 112, 243, 96, 68, 3, 201, 238, 80, 378, 185, 8]).reshape(-1, 1)
length_of_stay = np.array([19, 20, 22, 31, 48, 33, 30, 25, 43, 24]).reshape(-1, 1)
flight_hour = np.array([7, 3, 17, 4, 15, 20, 6, 14, 12, 2]).reshape(-1, 1)
flight_duration = np.array([5.52, 5.07, 7.57, 6.62, 7., 4.75, 8.83, 7.42, 6.42, 5.33]).reshape(-1, 1)

scaler = StandardScaler()

num_passengers_scaled = scaler.fit_transform(num_passengers)
purchase_lead_scaled = scaler.fit_transform(purchase_lead)
length_of_stay_scaled = scaler.fit_transform(length_of_stay)
flight_hour_scaled = scaler.fit_transform(flight_hour)
flight_duration_scaled = scaler.fit_transform(flight_duration)




In [ ]:
for column in dataset.columns:
    unique_values = dataset[column].unique()
    unique_count = dataset[column].nunique()
    print(f"\n🔹 {column} ({unique_count} унікальних значень):")

    print(unique_values)



🔹 num_passengers (9 унікальних значень):
[ 0.40315017 -0.58059134  1.38689168  2.3706332   4.33811622  3.35437471
  5.32185773  7.28934076  6.30559925]

🔹 purchase_lead (470 унікальних значень):
[ 1.96082368e+00  3.01702542e-01  1.75066833e+00  1.24729621e-01
 -1.84972991e-01 -9.03925482e-01  1.28611442e+00  1.69536430e+00
 -5.22432998e-02  3.24387736e+00  1.10914149e+00 -8.48621445e-01
  1.99400610e+00  1.77278995e+00  1.18656715e+00  1.92764125e+00
 -7.26952561e-01 -1.96033798e-01  2.94523555e+00  1.58475622e+00
 -7.04830946e-01 -6.82709331e-01 -6.60587716e-01 -7.60134984e-01
 -2.07094606e-01 -3.73006719e-01 -5.27858025e-01  1.80033659e-01
 -8.59682252e-01 -8.81803867e-01 -5.72101255e-01 -7.15891754e-01
 -3.17702681e-01 -7.93317407e-01 -1.51790568e-01  2.88993151e+00
 -8.37560637e-01 -2.18155413e-01  1.50733057e+00  2.46398505e-01
 -7.82256599e-01 -8.54257225e-02 -6.71648523e-01 -4.17249949e-01
 -6.93770139e-01 -8.15439022e-01 -5.49979640e-01  2.07143175e+00
  1.82809399e+00  8.4368

# Звіт про очищення даних
Для підготовки набору даних підприємства до побудови моделей машинного навчання було виконано комплексну обробку даних, що включала очищення, трансформацію та кодування ознак. Нижче наведено детальний опис виконаних етапів:

# Очищення даних:

Перевірка відсутніх значень: Аналіз вихідного датасету показав, що пропущені значення відсутні. Таким чином, заміна пропущених значень не проводилася.
Видалення дублікатів: змееншення розмір датасету з 50,000 до 49,281 записів. Це забезпечило унікальність даних і уникнення зміщення моделей через повторювані записи.


# Кодування категоріальних ознак:

One-Hot Encoding: Застосовано до стовпців sales_channel та trip_type.

Стовпець sales_channel був перетворений у два бінарні стовпці: sales_channel_Internet та sales_channel_Mobile.
Стовпець trip_type був перетворений у три бінарні стовпці: trip_type_RoundTrip, trip_type_CircleTrip, trip_type_OneWay.
Отримані бінарні стовпці були приведені до типу int для сумісності з алгоритмами машинного навчання.


Frequency Encoding: Застосовано до стовпців flight_day, route та booking_origin.

Стовпець flight_day був закодований частотою появи кожного дня тижня.
Стовпець route був закодований частотою появи маршрутів, що зменшило розмірність із потенційних 799 стовпців до одного числового стовпця.
Frequency encoding дозволило ефективно обробити категоріальні ознаки з великою кількістю унікальних значень, зберігаючи інформацію про їхню частоту в датасеті.




# Обробка аномальних значень:

Аналіз унікальних значень показав, що в стовпцях purchase_lead та length_of_stay є екстремальні значення. Однак ці значення не були видалені, оскільки вони можуть відображати реальні сценарії.
Стовпець length_of_stay містить значення 0, що може бути аномалією для певних типів поїздок. Однак без додаткового контексту це значення залишено, оскільки воно може бути дійсним для одноденних поїздок.


# Масштабування ознак:

Для числових ознак (num_passengers, purchase_lead, length_of_stay, flight_hour, flight_duration) застосовано StandardScaler для стандартизації даних. Це забезпечило нормалізацію розподілу даних, що є важливим для алгоритмів машинного навчання, таких як логістична регресія.


# Обґрунтування методів інженерії даних
One-Hot Encoding:

Обґрунтування: Використання one-hot encoding для sales_channel (2 категорії) та trip_type (3 категорії) було доцільним, оскільки ці ознаки мають малу кількість унікальних значень. One-hot encoding дозволяє уникнути штучного порядку між категоріями, що є важливим для алгоритмів машинного навчання, які не передбачають порядкових зв’язків. Перетворення в бінарні стовпці забезпечує коректне представлення даних для моделей.



Frequency Encoding:

Обґрунтування: Для ознак із великою кількістю унікальних значень (route — 799, booking_origin — 104, flight_day — 7) використано frequency encoding. Цей метод замінює категорії їхньою частотою появи в датасеті, що дозволяє зберегти інформацію про розподіл категорій без створення великої кількості нових стовпців, як у випадку one-hot encoding. Наприклад, для route frequency encoding зменшило розмірність із потенційних 799 стовпців до одного.


#Висновок:

Виконана підготовка даних включала видалення дублікатів, кодування категоріальних ознак (one-hot encoding для sales_channel і trip_type, frequency encoding для flight_day, route і booking_origin), обробку аномалій через стандартизацію числових ознак. Ці методи були обрані для забезпечення компактного представлення даних, зменшення розмірності, збереження інформаційного змісту та підготовки датасету до ефективного використання в моделях машинного навчання.
